# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**⚠️ Make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating) before starting with this lab.** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [4]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [5]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [3]:
def preprocess(text):
    # Tokenize
    tokens = nlp(text)
    
    # Lemmatisation + stop word and non-alphabetical filtering
    result = []
    for word in tokens:
        if ((not word.is_stop) and word.is_alpha):
            result.append(word.lemma_)
            
            
    return result

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses the given text by tokenising it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

To speed up the preprocessing, you should disable loading those spaCy components you do not need, such as the parser, and named entity recogniser. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

### 🤞 Test your code

Test your implementation by running the following cell:

In [4]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer = preprocess)
vector = vectorizer.fit_transform(df['description'])

### 🤞 Test your code

Test your implementation by running the following cell:

In [127]:
vector.shape

(1614, 21396)

This should show the dimensions of the matrix `X` to be around 1614 × 21389. (The number of rows should be exactly 1614. The number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.)

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [5]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def search(query):
   
    neigh = NearestNeighbors(metric = 'cosine')
    neigh.fit(vector)
    
    V_query = vectorizer.transform([query])
    
    return df.loc[neigh.kneighbors(V_query, 10)[1][0].tolist()]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

### 🤞 Test your code

Test your implementation by running the following cell:

In [20]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
1153,Road Riot,Road Riot is the global sensation that defined...


The top hit in the list should be *Subway Surfers*.

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [26]:
terms = vectorizer.get_feature_names_out()
idf_values = vectorizer.idf_

# Create a pandas dataframe
d = {'term': terms, 'IDF': idf_values}
term_df = pd.DataFrame(data = d)
term_df = term_df.sort_values(by=['IDF', 'term'], ascending=[True, True])

## Problem 4: Finding terms with low/high idf

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [27]:
print(term_df[0:10])
print(term_df[len(term_df)-10:len(term_df)])

          term       IDF
6106      game  1.297013
11956     play  1.420123
5428   feature  1.472359
5852      free  1.584695
10639      new  1.665665
17822    world  1.781792
16081     time  1.810621
692        app  1.838871
5953       fun  1.859132
16953      use  1.873860
        term       IDF
21385  회원가입에  7.693943
21386    회원을  7.693943
21387    획득한  7.693943
21388     효과  7.693943
21389    효과음  7.693943
21391    ﬁnd  7.693943
21392  ﬁnger  7.693943
21393  ﬁnish  7.693943
21394   ﬁrst  7.693943
21395    ﬂye  7.693943


### Result explanation

We can see that the terms with the highest idf are korean terms and english terms. Since most of the apps in the dataset have english descriptions, it is not strange that the korean terms are so high. What is more unusual are the english terms, but just means that these terms appear rarely across all the documents.
On the other end, the terms with the lowest idf are terms that could often appear in an apps description, such as game, app, new, free, etc.

## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [28]:
def keywords(text, n=10):
    tf_idf_values = vectorizer.transform([text]).toarray()[0]
    
    # Convert data to pandas df for sorting
    d = {'term': terms, 'TF-IDF': tf_idf_values}
    keywords_df = pd.DataFrame(data = d)
    keywords_df = keywords_df.sort_values(by=['TF-IDF', 'term'], ascending=[False, True])

    return keywords_df[0:n]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

### 🤞 Test your code

Test your implementation by running the following cell:

In [29]:
print(keywords(df['description'][1428], 10))

             term    TF-IDF
16278       train  0.365161
12693    railroad  0.209714
12694     railway  0.209714
12692        rail  0.193471
2537        chaos  0.174781
3359        crash  0.155508
6900         haul  0.122324
9187   locomotive  0.122324
11318    overcast  0.122324
12695    railyard  0.122324


This should give output similar to the following:

The exact output may differ slightly depending on the strategy used to break ties.

## Reflection questions

The following reflection questions will help you prepare for the diagnostic test. Answer each of them in the form of a short text and put your answers in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *application-specific* stop words?

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

**1.1**: We remove stop words since they are words that do not hold any relevant information about the text (words like the, a, they do not tell us anything interesting about a document). Words are lemmatised so that we can minimize the size of our list of our terms, since different inflections of a term do not really hold any more information than the base term. 
An application specific stop word would be the term "game" when searching a list of game review documents.

**1.2**: The first dimension (=1614) represents all the app samples, while the second dimension (=21396) represents the number of unique terms that were discovered. This matrix contains the tf-idf values for every term for every document. This means that the matrix represents how rare a term occurs in all the documents and how often it occurs in a specific document. This information is useful when searching for relevant documents but results in information such as total number of occurrences of a term in a document being lost.

**1.3**: A high idf value means that a term rarely appears in a set of documents while a low value means that a term appears often in a set of documents. As such one can use high idf terms to identify potential stop words in a set of documents.
Idf is not used on its own because it does not value documents with higher use of a term any  different compared to all other documents with lower use of the term. Ex. If you search with the terms "apple" and "fish", it would give document_1 (with 54 uses of apple and 76 uses of fish) and document_2 (with 1 use of apple and 3 uses of fish) the same weight (relevance). That is the reason idf is paired with tf, so that document_1 will be given a higher weight than document_2.


**Congratulations on finishing L1! 👍**